<a href="https://colab.research.google.com/github/amoghantarkar/galileo.ai/blob/main/wild_fires_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote
from urllib.error import HTTPError
from zipfile import ZipFile

CHUNK_SIZE = 40960 
DATASET_MAPPING = 'australian-bush-fire-satellite-data-nasa:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F495798%2F920571%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20201012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20201012T042520Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D62a925e60e142c6396bbb8da491eafaf2f032a6a24c46a9086c9c4f3846d4cffa640d1e92c624dcf9dabfd0aa04601c577acc9d910b8f5329b687b3442f4e595ef5a1d65ca859436eae778b2dfe0c2613479c632543c1c3afbf11699d1a564718e6e8a10f3129c2c10214cf094cec1076b92a91b7e668a3477d6370454560dfc497cb188c3f358d011c93fae1bb1783ae03f751cdabf585624ad1604b9107ce9dab1cb3a2344ddfbf6ecdf219f37695a7a7afba9e9cdc24c9d71979d24f0bc99982827c7cadd4b15d5df563a3ce83363b29667ddcd702969292c74401b585f88112d6af8f73690fdba16d348568edf42e7221e1c2ce281d151db955bbff70ac7,indian-wildfire-nasa-dataset-8-years:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F564667%2F1066790%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20201012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20201012T042520Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D60f6b07cb4e4b70fb00c62e84e3400af3e7fb97b0c75d1110a504e37ee10f9fed710f73dbfb9bc290945b8f25c7eec245b116bcebb948e5071fefd35040f0475780486fc056e5c848f94ffca1b32a37ce1268d8a5233ce38d7bbfb7fd3d32e4aa8144d39930b71a6b6ebd37a6583f02747276ec2fccb97b8fa792dbd2351bf954f5d2f656650a3feec687b636ea47fe53588367d441ba731584d512409baeab43b28788600494be4514d42d72f01c8476c792d05f9862d25403b9e04c7837150875ea675d85a9815cc0aaa4cccb3c9a421084f3f75973f96eb480f2adf835291ff75edea1f627c872edb9c3267502bb2d3f135934f37ad015784c83730db1d50'
KAGGLE_INPUT_PATH='/home/kaggle/input'
KAGGLE_INPUT_SYMLINK='/kaggle'

!mkdir -p -- $KAGGLE_INPUT_PATH
!chmod 777 $KAGGLE_INPUT_PATH
!ln -sfn $KAGGLE_INPUT_PATH ../
!mkdir -p -- $KAGGLE_INPUT_SYMLINK
!ln -sfn $KAGGLE_INPUT_PATH $KAGGLE_INPUT_SYMLINK

for dataset_mapping in DATASET_MAPPING.split(','):
    directory, download_url_encoded = dataset_mapping.split(':')
    download_url = unquote(download_url_encoded)
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
            total_length = zipfileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes zipped')
            dl = 0
            data = zipfileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = zipfileres.read(CHUNK_SIZE)
            print(f'\nUnzipping {directory}')
            with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue
print('Dataset import complete.')


!pip install git+https://github.com/python-visualization/branca
!pip install git+https://github.com/sknzl/folium@update-css-url-to-https

#dependencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #plotting
import seaborn as sns #for beatiful visualization
import folium
from folium import plugins

#set file path
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



fire_nrt_m6 = pd.read_csv("../input/indian-wildfire-nasa-dataset-8-years/fire_nrt_M6_107977.csv")
fire_archive_m6 = pd.read_csv("../input/indian-wildfire-nasa-dataset-8-years/fire_archive_M6_107977.csv")
fire_nrt_v1 = pd.read_csv("../input/indian-wildfire-nasa-dataset-8-years/fire_nrt_V1_107978.csv")
fire_archive_v1 = pd.read_csv("../input/indian-wildfire-nasa-dataset-8-years/fire_archive_V1_107978.csv")
fire_nrt_v2 = pd.read_csv("../input/australian-bush-fire-satellite-data-nasa/fire_nrt_V1_101674.csv")
fire_archive_v2 = pd.read_csv("../input/australian-bush-fire-satellite-data-nasa/fire_archive_V1_101674.csv")

# type(fire_nrt_v1)

df_filter = data.filter(["latitude","longitude","acq_date","frp"])
df_filter.head()

df = df_filter[df_filter['acq_date']>='2015-11-01']
df.count()

data_topaffected = df.sort_values(by='frp',ascending=False).head(200)
data_topaffected


#Create a map

mapbox_api_key = r"pk.eyJ1IjoiYW1vZ2hhbnRhcmthciIsImEiOiJja2c1cHJlMjgwMXd3Mnpxdm9lbGY4aHp5In0.KMFMRgjtKlU-CzMdb5IgAQ" 
tileset_ID_str = "satellite-streets-v11";#mapbox://styles/mapbox/satellite-streets-v11
tilesize_pixels = "256"; 
m = folium.Map(location=[28.644800, 77.216721], zoom_start=3, 
                tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}", 
                attr="Mapbox")
# m = folium.Map(location=[28.644800, 77.216721], control_scale=True, zoom_start=2,attr = "text some")
df_copy = data_topaffected.copy()

# loop through data to create Marker for each hospital
for i in range(0,len(df_copy)):
    
    folium.Marker(
    location=[df_copy.iloc[i]['latitude'], df_copy.iloc[i]['longitude']],
    #popup=popup,
    tooltip="frp: " + str(df_copy.iloc[i]['frp']) + "<br/> date: "+ str(df_copy.iloc[i]['acq_date']),
    icon=folium.Icon(color='red',icon='fire',prefix="fa"),
    ).add_to(m)
        
m

from folium.plugins import HeatMapWithTime
# A small function to get heat map with time given the data

def getmap(ip_data,location,zoom,radius):
    
    #get day list
    dfmap = ip_data[['acq_date','latitude','longitude','frp']]
    df_day_list = []
    for day in dfmap.acq_date.sort_values().unique():
        df_day_list.append(dfmap.loc[dfmap.acq_date == day, ['acq_date','latitude', 'longitude', 'frp']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())
    
    mapbox_api_key = r"pk.eyJ1IjoiYW1vZ2hhbnRhcmthciIsImEiOiJja2c1cHJlMjgwMXd3Mnpxdm9lbGY4aHp5In0.KMFMRgjtKlU-CzMdb5IgAQ" 
    tileset_ID_str = "satellite-streets-v11";#mapbox://styles/mapbox/satellite-streets-v11
    tilesize_pixels = "256"; 
    m = folium.Map(location, zoom_start=zoom, 
                     tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}", 
                     attr="Mapbox")
    # Create a map using folium
#     m = folium.Map(location, zoom_start=zoom,tiles='Stamen Terrain')
    #creating heatmap with time
    HeatMapWithTime(df_day_list,index =list(dfmap.acq_date.sort_values().unique()), auto_play=False,radius=radius, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(m)

    return m
m=getmap(df,[28.644800, 77.216721],3.5,2)
m
